**Защита персональных данных клиентов**

Имеются данные клиентов страховой компании. Нам необходимо создать метод преобразования данных для сохранения конфиденцильности имеющейся информации при работе с моделью машинного обучения. Качество работы модели после внесеннных преобразований не должно пострадать.

**1  Анализ данных**

In [1]:
import numpy as np
import pandas as pd
from random import randint
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
display(data.head())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
print(data.isna().sum())

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64


In [4]:
data[['Возраст', 'Зарплата']] = data[['Возраст', 'Зарплата']].astype(int)
print(data.info)

<bound method DataFrame.info of       Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0       1       41     49600            1                  0
1       0       46     38000            1                  1
2       0       29     21000            0                  0
3       0       21     41700            2                  0
4       1       28     26100            0                  0
...   ...      ...       ...          ...                ...
4995    0       28     35700            2                  0
4996    0       34     52400            1                  0
4997    0       20     33900            2                  0
4998    1       22     32700            3                  0
4999    1       28     40600            1                  0

[5000 rows x 5 columns]>


**2  Умножение матриц**

Обозначения:

𝑋  — матрица признаков (нулевой столбец состоит из единиц)

𝑦  — вектор целевого признака

𝑃  — матрица, на которую умножаются признаки

𝑤  — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

𝑎=𝑋𝑤
 
Задача обучения:

𝑤=argmin𝑤𝑀𝑆𝐸(𝑋𝑤,𝑦)
 
Формула обучения:

𝑤=(𝑋𝑇𝑋)−1𝑋𝑇𝑦

Ответ: не изменится.

Обоснование:

Рассмотрим предсказания для измененной матрицы признаков.  а1=𝑋∗𝑃∗𝑤1 
𝑎1=𝑋∗𝑃∗((𝑋∗𝑃)𝑇∗(𝑋∗𝑃))−1∗(𝑋∗𝑃)𝑇∗𝑦 
Обратное значение произведения обратимых матриц A и B с одинаковым числом строк и столбцов будет равно значению, полученному при умножении матриц, обратных исходным, то есть  (𝐴×𝐵)−1=𝐵−1×𝐴−1 . Для нашего решения:

𝑎1=𝑋∗𝑃∗(𝑋∗𝑃)−1∗((𝑋∗𝑃)𝑇)−1∗(𝑋∗𝑃)𝑇∗𝑦 
𝑎1=𝑋∗𝑃∗𝑃−1∗𝑋−1∗((𝑋∗𝑃)𝑇)−1∗(𝑋∗𝑃)𝑇∗𝑦 
При умножении обратной матрицы на исходную матрицу получается единичная матрица.

𝑎1=𝑋∗𝐸∗𝑋−1∗((𝑋∗𝑃)𝑇)−1∗(𝑋∗𝑃)𝑇∗𝑦 
Произведение любой матрицы и единичной матрицы подходящего размера равно самой матрице.

𝑎1=𝑋∗𝑋−1∗((𝑋∗𝑃)𝑇)−1∗(𝑋∗𝑃)𝑇∗𝑦 
Транспонирование произведения двух матриц эквивалентно произведению транспонированных матриц, взятых в обратном порядке:  (А∗В)𝑇=В𝑇∗А𝑇 . Используем это в нашем расчёте.

𝑎1=𝑋∗𝑋−1∗(𝑃𝑇∗𝑋𝑇)−1∗𝑃𝑇∗𝑋𝑇∗𝑦 
Ещё раз используем свойство обратных матриц.

𝑎1=𝑋∗𝑋−1∗(𝑋𝑇)−1∗(𝑃𝑇)−1∗𝑃𝑇∗𝑋𝑇∗𝑦 
Вновь получаем единичную матрицу.

𝑎1=𝑋∗𝑋−1∗(𝑋𝑇)−1∗𝐸∗𝑋𝑇∗𝑦 
Произведение любой матрицы и единичной матрицы подходящего размера равно самой матрице.

𝑎1=𝑋∗𝑋−1∗(𝑋𝑇)−1∗𝑋𝑇∗𝑦 
Используем свойство обратной матрицы.

𝑎1=𝑋∗((𝑋𝑇)∗𝑋)−1∗𝑋𝑇∗𝑦 
Расчетное значение формулы обучения:  𝑤=(𝑋𝑇𝑋)−1∗𝑋𝑇∗𝑦 , подставляем в полученное равенство.

𝑎1=𝑋∗𝑤 .

𝑎1=𝑎 
Получаем, что не зависимо, от изменения матрицы признаков, предсказния будут расччитаны так же. Качество предсказаний не изменится.

**3  Алгоритм преобразования**

Ранее мы уже рассмотрели, что качество линейной регрессии при домножении имеющейся матрицы признаков на обратимую матрицу не изменится. Соответственно, данное качество можно использовать для защиты данных пользователей. Создадим некий ключ - обратимую матрицу, значения имеющейся таблицы признаков умножим на неё, чтобы скрыть данные. Для дешифровки данных необходимо будет имеющуюся матрицу с данными разделить на ключ. В случае с работы с матрицами, обращаем внимание на то, что деление матрицы на матрицу невозможно, а потому будет происходить умножение на матрицу, обратную ключу.

**4  Проверка алгоритма**

Создаем матрицу key - ключ для шифрования матрицы признаков.

In [5]:
key = np.random.randint(0, 20, (4, 4))
print(key)

[[ 7 18 18 17]
 [18 15 11 12]
 [ 4  1 17  1]
 [ 4  0 16 13]]


Из имеющейся выборки выделим целевой признак.

In [6]:
features = data.drop(['Страховые выплаты'], axis = 1)
target = data['Страховые выплаты']
features_m = features.values
target_m = target.values
print(features_m)

[[    1    41 49600     1]
 [    0    46 38000     1]
 [    0    29 21000     0]
 ...
 [    0    20 33900     2]
 [    1    22 32700     3]
 [    1    28 40600     1]]


Создадим зашифрованную матрицу с данными признаков.

In [7]:
features_encr = np.dot(features_m, key)
print(features_encr)

[[199149  50233 843685  50122]
 [152832  38690 646522  38565]
 [ 84522  21435 357319  21348]
 ...
 [135968  34200 576552  34166]
 [131215  33048 556208  33020]
 [162915  41038 690542  40966]]


In [8]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550308169177


In [9]:
model.fit(features_encr, target)
predictions_2 = model.predict(features_encr)
print(r2_score(target, predictions_2))

0.4249455030816913


Проверка показывает, что качество работы модели линейной регрессии не зависит от произведенных манипуляций с матрицей признаков. Величина метрики качества R2 для незашифрованных данных - 0.42494550308169177, для зашифрованных - 0.42494550308167944, практически идентична.

**Заключение**

Нам были предоставлены данные клиентов страховой компании. Необходимо было создать метод преобразования данных, для их защиты и сохранения при использовании моделей машинного обучения. Качество работы модели линейной регрессии при работы с данными до преобразования и после должны оставаться неизменным.

Для решения поставленной задачи было обосновано свойство модели регрессии, что при изменении матрицы признаков из имеющихся данных, величина предсказаний не изменяется, а значит и качество работы модели остается на прежнем уровне.

В соответствии с полученным обоснованием составляем алгоритм действий по преобразованию предоставленных данных. Матрица признаков умножается на матрицу – ключ для шифрования данных.

После проведения указанных изменений проводим проверку работы модели линейной регрессии на данных до преобразования и после. Метрика для проверки качества модели – R2. Рассчитанная величина качества модели в обоих случаях одинакова, шифрование данных не помешало модели линейной регрессии работать с прежней точностью.

Данное свойство работы модели позволяет использовать зашифрованные данные пользователей для предсказаний необходимых данных, сохраняя тем самым конфиденциальность используемой информации.